# import

In [14]:
from Database.Database import DataFunFilm
import pandas as pd
from gensim.models import Word2Vec
from gensim.test.utils import common_texts
import gensim.downloader as api
import gensim
import pyemd
from preProc import PrepText
import re
import numpy as np

# Загрузка базы данных

In [113]:
db = DataFunFilm()
films = db.getTableDF()
films = films.head(1000)
films

,id,kinopoisk_id,imdb_id,name_ru,name_original,poster_url,rating_imdb,rating_imdb_vote_count,rating_film_critics,rating_critics_vote_count,year,film_length,genres,web_url,slogan,description,rating_kinopoisk,short_desription,rating_age_limits
0,1,1043658,tt4202274,Шоу Грязного Фрэнка,The Filthy Frank Show,https://kinopoiskapiunofficial.tech/images/pos...,9.7,32929,NaN,0,2011.0,12.0,фэнтези комедия музыка короткометражка,https://www.kinopoisk.ru/series/1043658/,None,Шоу повествует о героической борьбе сэра Фрэнс...,9.3,None,None
1,2,1309325,None,Космос,,https://kinopoiskapiunofficial.tech/images/pos...,NaN,0,NaN,0,2019.0,NaN,документальный,https://www.kinopoisk.ru/series/1309325/,None,Астрофизик Нил Деграсс Тайсон встает за штурва...,9.3,Нил Деграсс Тайсон отправляется в мысленное пу...,age18
2,3,77044,tt0108778,Друзья,Friends,https://kinopoiskapiunofficial.tech/images/pos...,9.0,937875,NaN,0,1994.0,22.0,мелодрама комедия,https://www.kinopoisk.ru/series/77044/,Everybody needs Friends!,"Шестеро друзей – Рейчел, Моника, Фиби, Джоуи, ...",9.2,"Шестеро друзей, две квартиры и много юмора. Си...",age16
3,4,972889,tt4926348,BadComedian,None,https://kinopoiskapiunofficial.tech/images/pos...,7.7,793,NaN,0,2011.0,NaN,комедия,https://www.kinopoisk.ru/series/972889/,None,"Юмористическое шоу, высмеивающее плохое кино. ...",9.2,None,age18
4,5,674243,None,Счастливые люди,None,https://kinopoiskapiunofficial.tech/images/pos...,8.8,120,NaN,0,2008.0,55.0,документальный,https://www.kinopoisk.ru/series/674243/,None,Документальный фильм о трудах и днях охотников...,9.2,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,1252445,tt13499372,Купала,None,https://kinopoiskapiunofficial.tech/images/pos...,NaN,0,NaN,0,2020.0,150.0,драма биография история,https://www.kinopoisk.ru/film/1252445/,None,Фильм повествует о драматичной судьбе народног...,8.2,None,None
996,997,79741,tt0108757,Скорая помощь,ER,https://kinopoiskapiunofficial.tech/images/pos...,7.9,58338,NaN,0,1994.0,43.0,драма мелодрама,https://www.kinopoisk.ru/series/79741/,A true medical marvel,Герои остросюжетного сериала «Скорая помощь» -...,8.2,None,age16
997,998,94296,None,Освобождение: Огненная дуга,None,https://kinopoiskapiunofficial.tech/images/pos...,7.5,363,NaN,0,1968.0,93.0,драма мелодрама военный,https://www.kinopoisk.ru/film/94296/,Two great armies duel to the death with a mill...,Масштабная историческая киноэпопея Юрия Озеров...,8.2,"1943 год, битва на Курской дуге, предопределив...",age12
998,999,44842,None,Когда наступает сентябрь...,None,https://kinopoiskapiunofficial.tech/images/pos...,7.6,103,NaN,0,1975.0,91.0,драма семейный,https://www.kinopoisk.ru/film/44842/,None,Пенсионер Левон Погосян прилетел в Москву из А...,8.2,None,age0


In [16]:
rus_words = gensim.downloader.load('word2vec-ruscorpora-300')

In [114]:
prep = PrepText()


Loading the model...


In [115]:
prepDisc = []
j = 0
for i in films["description"]:
    prepDisc.append(prep.tag_ud(i).split())
    j = j + 1
    print(j)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [116]:
films["prepDisc"] = prepDisc

In [117]:
films.head()

,id,kinopoisk_id,imdb_id,name_ru,name_original,poster_url,rating_imdb,rating_imdb_vote_count,rating_film_critics,rating_critics_vote_count,year,film_length,genres,web_url,slogan,description,rating_kinopoisk,short_desription,rating_age_limits,prepDisc
0,1,1043658,tt4202274,Шоу Грязного Фрэнка,The Filthy Frank Show,https://kinopoiskapiunofficial.tech/images/pos...,9.7,32929,NaN,0,2011.0,12.0,фэнтези комедия музыка короткометражка,https://www.kinopoisk.ru/series/1043658/,None,Шоу повествует о героической борьбе сэра Фрэнс...,9.3,None,None,"[шоу_PROPN, повествовать_VERB, о_ADP, героичес..."
1,2,1309325,None,Космос,,https://kinopoiskapiunofficial.tech/images/pos...,NaN,0,NaN,0,2019.0,NaN,документальный,https://www.kinopoisk.ru/series/1309325/,None,Астрофизик Нил Деграсс Тайсон встает за штурва...,9.3,Нил Деграсс Тайсон отправляется в мысленное пу...,age18,"[астрофизик_NOUN, нил_VERB, деграсс::тайсон_PR..."
2,3,77044,tt0108778,Друзья,Friends,https://kinopoiskapiunofficial.tech/images/pos...,9.0,937875,NaN,0,1994.0,22.0,мелодрама комедия,https://www.kinopoisk.ru/series/77044/,Everybody needs Friends!,"Шестеро друзей – Рейчел, Моника, Фиби, Джоуи, ...",9.2,"Шестеро друзей, две квартиры и много юмора. Си...",age16,"[шестеро_NUM, друг_NOUN, рейчел_PROPN, моника_..."
3,4,972889,tt4926348,BadComedian,None,https://kinopoiskapiunofficial.tech/images/pos...,7.7,793,NaN,0,2011.0,NaN,комедия,https://www.kinopoisk.ru/series/972889/,None,"Юмористическое шоу, высмеивающее плохое кино. ...",9.2,None,age18,"[юмористический_ADJ, шоу_NOUN, высмеивать_VERB..."
4,5,674243,None,Счастливые люди,None,https://kinopoiskapiunofficial.tech/images/pos...,8.8,120,NaN,0,2008.0,55.0,документальный,https://www.kinopoisk.ru/series/674243/,None,Документальный фильм о трудах и днях охотников...,9.2,None,None,"[документальный_ADJ, фильм_NOUN, о_ADP, труд_N..."


In [181]:
def moreSimilar(discripts, id):
    min = -1
    fil = -1
    disc = discripts[id]
    for i in range(len(discripts)):
        if i != id:
            d = rus_words.wmdistance(discripts[i], disc)
            if min == -1:
                min = d
                fil = i
            elif min > d:
                min = d
                fil = i
    print(min)
    return fil

In [182]:
moreSimilar(list(films["prepDisc"]),8)

1.0209330199712128


648

In [183]:
films.loc[films.id == 9]

,id,kinopoisk_id,imdb_id,name_ru,name_original,poster_url,rating_imdb,rating_imdb_vote_count,rating_film_critics,rating_critics_vote_count,year,film_length,genres,web_url,slogan,description,rating_kinopoisk,short_desription,rating_age_limits,prepDisc
8,9,326,tt0111161,Побег из Шоушенка,The Shawshank Redemption,https://kinopoiskapiunofficial.tech/images/pos...,9.3,2551973,8.4,81,1994.0,142.0,драма,https://www.kinopoisk.ru/film/326/,Страх - это кандалы. Надежда - это свобода,Бухгалтер Энди Дюфрейн обвинён в убийстве собс...,9.1,Несправедливо осужденный банкир готовит побег ...,age16,"[бухгалтер_NOUN, энди::дюфрейн_PROPN, обвинить..."


In [184]:
films.loc[films.id == moreSimilar(list(films["prepDisc"]),8)+1]

1.0209330199712128


,id,kinopoisk_id,imdb_id,name_ru,name_original,poster_url,rating_imdb,rating_imdb_vote_count,rating_film_critics,rating_critics_vote_count,year,film_length,genres,web_url,slogan,description,rating_kinopoisk,short_desription,rating_age_limits,prepDisc
648,649,1172552,tt8540754,Дикие скричеры,Screechers Wild!,https://kinopoiskapiunofficial.tech/images/pos...,7.1,20,NaN,0,2018.0,NaN,боевик мультфильм,https://www.kinopoisk.ru/series/1172552/,None,В недалеком будущем все сходят с ума по скриче...,8.3,Два брата мечтают победить в гонках машинок-тр...,age6,"[в_ADP, недалекий_ADJ, будущее_NOUN, все_PRON,..."
